In [ ]:
import datetime as dt
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd

In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
from flask import Flask, jsonify

In [ ]:
# Database
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect the tables
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
session = Session(engine)

In [ ]:
# Set up Flask and landing page
app = Flask(__name__)

In [ ]:
# last 12 months
last_twelve_months = '2016-08-23'

In [ ]:
# Convert the query results from your precipitation analysis
@app.route("/")
def welcome():
    return (
        f"<p>Welcome to the Hawaii weather API!</p>" 
    )

In [ ]:
# Return the JSON representation of your dictionary.
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Date 12 months ago
    p_results = session.query(Measurement.date, func.avg(Measurement.prcp)).filter(Measurement.date >= last_twelve_months).group_by(Measurement.date).all()
    return jsonify(p_results)

In [ ]:
# Return a JSON list of Temperature Observations (tobs) for the previous year
@app.route("/api/v1.0/tobs")
def tobs():
    t_results = session.query(Measurement.date, Measurement.station, Measurement.tobs).filter(Measurement.date >= last_twelve_months).all()
    return jsonify(t_results)

In [ ]:
# Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.
# TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.
@app.route("/api/v1.0/<date>")
def startDateOnly(date):
    day_temp_results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >= date).all()
    return jsonify(day_temp_results)

In [ ]:
# For a specified start date and end date, calculate TMIN, TAVG, and TMAX for the dates from the start date to the end date, inclusive.
@app.route("/api/v1.0/<start>/<end>")
def startDateEndDate(start,end):
    multi_day_temp_results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    return jsonify(multi_day_temp_results)

In [ ]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)